In [ ]:
%load_ext autoreload

%autoreload 2

In [ ]:
from environment import *

from setting import *

In [ ]:
path_dict = path(
    TITLE,
    ELEMENTS,
    NMF_K,
    WT_HCC_K,
    H_HCC_K,
    UPLOAD_TO_PLOTLY,
)

In [ ]:
feature_x_sample = ccal.read_and_process_feature_x_sample(
    FEATURE_X_SAMPLE_FILE_PATH,
    nanize=NANIZE,
    drop_na_axis=DROP_NA_AXIS,
    min_n_not_na_unique_value=MIN_N_NOT_NA_UNIQUE_VALUE,
    max_na=MAX_NA,
    shift_as_necessary_to_achieve_min_before_logging=SHIFT_AS_NECESSARY_BEFORE_LOGGING,
    log_base=LOG_BASE,
    normalization_axis=NORMALIZATION_AXIS,
    normalization_method=NORMALIZATION_METHOD,
    plot=False,
)

In [ ]:
if SELECT_GENE_SYMBOL:
    
    feature_x_sample = feature_x_sample.loc[feature_x_sample.index & set(ccal.select_gene_symbol(
        gene_family_name_to_remove=(
            '18S ribosomal RNAs',
            '28S ribosomal RNAs',
            '45S pre-ribosomal RNAs',
            '5.8S ribosomal RNAs',
            '5S ribosomal RNAs',
            'Cytoplasmic transfer RNAs',
            'Long non-coding RNAs (published)',
            'MicroRNAs',
            'Mitochondrially encoded ribosomal RNAs',
            'Mitochondrially encoded tRNAs',
            'Nuclear-encoded mitochondrial transfer RNAs',
            'Piwi-interacting RNA clusters',
            'Ribosomal 45S RNA clusters',
            'Ribosomal 45S rRNA genes outside of clusters',
            'RNAs, 7SL, cytoplasmic',
            'RNAs, Ro-associated Y',
            'Small Cajal body-specific RNAs',
            'Small NF90 (ILF3) associated RNAs',
            'Small nuclear RNAs',
            'Small nucleolar RNAs, C/D box',
            'Small nucleolar RNAs, H/ACA box',
            'Vault RNAs',
            'L ribosomal proteins',
            'Mitochondrial ribosomal proteins',
            'S ribosomal proteins',
            'Mitochondrial complex II: succinate dehydrogenase subunits',
            'Mitochondrial complex III: ubiquinol-cytochrome c reductase complex subunits',
            'Mitochondrial complex IV: cytochrome c oxidase subunits',
            'Mitochondrial complex V: ATP synthase subunits',
            'NADH:ubiquinone oxidoreductase core subunits',
            'NADH:ubiquinone oxidoreductase supernumerary subunits',
        ),
        locus_type_to_keep=('gene with protein product', ),
    ))]

In [ ]:
feature_x_sample.to_csv(
    path_dict['feature_x_sample_file_path'],
    sep='\t',
)

ccal.summarize_feature_x_sample(feature_x_sample)

feature_x_sample